This example is an MB-MVPA code that uses part of the data (Tom et al., 2007).<br>
Tom's full dataset used 16 subjects, but in this example, only 2 subjects are used to save resource.

### Import the MB-MVPA libarary.

Other libraries(nilean, keras, etc..) dosen't need to be imported.<br>
Because mb-mvpa has wrapping the libararies.<br>
You don't necessarily have to know fMRI libraries like nilearn and machine learning libraries like tensorflow.<br>
<b>MB-MVPA is all you need.</b>

Most of mb-mvpa are wrapping nilearn, tensorflow, Keras and etc., so warning can occur from that libraries.<br>
This page does not print warning because most of them are can be ignored.<br>
<b> You do not need to remove the warning when you are actually using it. </b>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from mbmvpa.preprocessing.bids import bids_preprocess
from mbmvpa.preprocessing.events import events_preprocess
from mbmvpa.data.loader import prepare_dataset
from mbmvpa.utils.example_utils import load_example_data

Data download from AWS S3, ~ 1GB.<br>
If you run this example more than once, the data will already be downloaded.<br>
If so, do not download it again.

In [3]:
root = load_example_data("tom")

data load success! (tom)


### Preprocessing bids data including fMRI images
The MB-MVPA liabrary need to BIDS format including preprocessed nii images*(nii or nii.gz)* and behavior data file*(.tsv)*.<br>
The fMRI images are usually located here<br>
**derivatives/fmriprep/**<br>
And the behavior data are located here<br>
**subject/session/run/func**

In [4]:
X, voxel_mask, layout, data_root = bids_preprocess(root, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=2, nthread=4)

bids preprocessing done!                          : 100%|██████████| 6/6 [00:19<00:00,  3.25s/it]  


<b>X</b> (numpy.array): subject-wise & run-wise BOLD time series data. shape : subject # x run # x timepoint # x voxel #.<br>
<b>voxel_mask</b> (nibabel.Nifti1Image): a nifti image for voxel-wise binary mask (ROI mask)

Our mbmvpa is directly inputted into the data and modulation functions to be used to increase user's freedom and increase stability.<br>
The following three functions must be completed, each of which means:<br>
1. If you use **hbayesDM**, you should change the columns name to suit it. -> preprocess
2. If you only want to use behavioral data under certain conditions, you must define a condition function. -> condition
3. Finally, since we are model-based fMRI, we have to calculate the modulation.<br>
   The important point is that the behavioral data that we already have are in the **"row"**, and parameters estimated by model are in the **"param_dict"**.<br>
   -> modulation

''' python
'''

If you have parameters already calculated and can provide them in *.tsv*, we also allow that.<br>
"**individual_params_custom"** has the role.<br>
In that case, the dm_model, or deicison making model, is not required.

In [5]:
def example_tom_modulation(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row

In [6]:
dm_model, df_events, signals, time_masks, _ = \
    events_preprocess(root,
                      modulation=example_tom_modulation,
                      individual_params_custom="../example_data/tom_example/derivatives/fmriprep/mvpa/individual_params.tsv")

calculating modulation..                          :  83%|████████▎ | 5/6 [00:02<00:00,  1.23it/s]  INFO:numexpr.utils:Note: detected 88 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 88 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
events preproecssing done!                        : : 7it [00:03,  2.25it/s]                     


### Load data and shape check

Through the above process, **X(fMRI images)** and **Y(modulation)** dataset can be obtained.<br>
Their shape[0] is must be equal.

In [7]:
X, y, voxel_mask = prepare_dataset(data_root)

### Fitting MVPA models & Results

In [8]:
from mbmvpa.utils.coef2map import get_map
from time import perf_counter

In [ ]:
from mbmvpa.models.regressor import penalized_linear_regression

s = perf_counter()
coefs = penalized_linear_regression(X, y,
                                    layout,
                                    lambda_param=2.0,
                                    batch_size=256,
                                    N=10,
                                    verbose=1)
result = get_map(coefs, voxel_mask, task_name="tom2007_penalized_linear", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

INFO:root:[1/10] - val_mse: 0.1820
INFO:root:[2/10] - val_mse: 0.1726
INFO:root:[3/10] - val_mse: 0.1606
INFO:root:[4/10] - val_mse: 0.1851
INFO:root:[5/10] - val_mse: 0.1644
INFO:root:[6/10] - val_mse: 0.1523
INFO:root:[7/10] - val_mse: 0.1453
INFO:root:[8/10] - val_mse: 0.1732
INFO:root:[9/10] - val_mse: 0.1685


In [ ]:
from mbmvpa.models.regressor import mlp_regression

s = perf_counter()
coefs = mlp_regression(X, y,
                       layout,
                       layer_dims=[1024, 1024],
                       activation="linear",
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=3,
                       verbose=1)
result = get_map(coefs, voxel_mask, task_name="tom2007_mlp", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

In [ ]:
from mbmvpa.models.regressor import elasticnet

s = perf_counter()
coefs = elasticnet(X, y,
                   layout,
                   n_jobs=16,
                   verbose=1,
                   max_lambda=1,
                   n_samples=5000)
result = get_map(coefs, voxel_mask, task_name="tom2007_elasticnet", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")